In [71]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/breast-cancer-wisconsin-data/data.csv


# Data preprocessing

In [33]:
data = pd.read_csv("/kaggle/input/breast-cancer-wisconsin-data/data.csv")

In [34]:
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [35]:
data.shape

(569, 33)

In [36]:
data.dropna()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32


In [37]:
data.dtypes

id                           int64
diagnosis                   object
radius_mean                float64
texture_mean               float64
perimeter_mean             float64
area_mean                  float64
smoothness_mean            float64
compactness_mean           float64
concavity_mean             float64
concave points_mean        float64
symmetry_mean              float64
fractal_dimension_mean     float64
radius_se                  float64
texture_se                 float64
perimeter_se               float64
area_se                    float64
smoothness_se              float64
compactness_se             float64
concavity_se               float64
concave points_se          float64
symmetry_se                float64
fractal_dimension_se       float64
radius_worst               float64
texture_worst              float64
perimeter_worst            float64
area_worst                 float64
smoothness_worst           float64
compactness_worst          float64
concavity_worst     

In [38]:
data.describe

<bound method NDFrame.describe of            id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0      842302         M        17.99         10.38          122.80     1001.0   
1      842517         M        20.57         17.77          132.90     1326.0   
2    84300903         M        19.69         21.25          130.00     1203.0   
3    84348301         M        11.42         20.38           77.58      386.1   
4    84358402         M        20.29         14.34          135.10     1297.0   
..        ...       ...          ...           ...             ...        ...   
564    926424         M        21.56         22.39          142.00     1479.0   
565    926682         M        20.13         28.25          131.20     1261.0   
566    926954         M        16.60         28.08          108.30      858.1   
567    927241         M        20.60         29.33          140.10     1265.0   
568     92751         B         7.76         24.54           47.92      181

In [39]:
data['diagnosis'].values

array(['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'B', 'B', 'B', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'B', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'B', 'M', 'B', 'B', 'B', 'B',
       'B', 'M', 'M', 'B', 'M', 'M', 'B', 'B', 'B', 'B', 'M', 'B', 'M',
       'M', 'B', 'B', 'B', 'B', 'M', 'B', 'M', 'M', 'B', 'M', 'B', 'M',
       'M', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'M', 'M', 'B', 'B', 'B',
       'M', 'B', 'B', 'M', 'M', 'B', 'B', 'B', 'M', 'M', 'B', 'B', 'B',
       'B', 'M', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'M', 'M', 'M', 'B', 'M', 'M', 'B', 'B', 'B', 'M', 'M', 'B', 'M',
       'B', 'M', 'M', 'B', 'M', 'M', 'B', 'B', 'M', 'B', 'B', 'M', 'B',
       'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'M', 'B', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'B', 'B', 'M', 'M',
       'B', 'B', 'M', 'M', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'M

In [40]:
data['diagnosis'] = data['diagnosis'].map({'M':0,'B': 1})

In [41]:
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [42]:
data.duplicated().sum()

0

In [43]:
data.isnull().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

# Splitting Dataset

In [50]:
x = data[['radius_mean' , 'texture_mean' , 'perimeter_mean' , 'area_mean']]
y = data['diagnosis']

In [51]:
x

,radius_mean,texture_mean,perimeter_mean,area_mean
0,17.99,10.38,122.80,1001.0
1,20.57,17.77,132.90,1326.0
2,19.69,21.25,130.00,1203.0
3,11.42,20.38,77.58,386.1
4,20.29,14.34,135.10,1297.0
...,...,...,...,...
564,21.56,22.39,142.00,1479.0
565,20.13,28.25,131.20,1261.0
566,16.60,28.08,108.30,858.1
567,20.60,29.33,140.10,1265.0


In [52]:
y

0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: diagnosis, Length: 569, dtype: int64

In [57]:
y = y.values.reshape(-1, 1) 

In [58]:
x_train,x_test,y_train,y_test= train_test_split(x,y,train_size=0.4,random_state=42)

# Selecting different kernel

## kernel = rbf (radial basis function)

In [59]:
reg = SVR(kernel='rbf',C=100 , gamma=0.1, epsilon=0.1)
reg.fit(x_train,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=100, gamma=0.1)

In [62]:
pred = reg.predict(x_test) # when using y_test it asks for 4 inputs

In [63]:
pred

array([0.72752736, 0.55961505, 0.48695905, 0.71244016, 0.79990612,
       0.56002221, 0.5600429 , 0.53834366, 0.86724072, 0.58942128,
       0.60600486, 0.55999993, 0.61886306, 0.56006751, 0.61079398,
       0.56000316, 0.48778296, 0.56006244, 0.5600429 , 0.56004287,
       0.70824051, 0.8093171 , 0.5600429 , 0.56004293, 0.59830036,
       0.64536585, 0.60664367, 0.56005104, 0.64411971, 0.48531424,
       0.76397874, 0.64539644, 0.73139994, 0.85732014, 0.56045717,
       0.85694643, 0.54346095, 0.70707205, 0.24820202, 0.30932632,
       0.86866595, 0.55992263, 0.75962075, 0.65227275, 0.81525277,
       0.72623882, 0.56010239, 0.56719513, 0.70206986, 0.75811236,
       0.54261376, 0.54891471, 0.37361383, 0.62024122, 0.64481261,
       0.78143937, 0.74223113, 0.5600429 , 0.94820542, 0.65790027,
       0.93898686, 0.41476389, 0.5600429 , 0.55297891, 0.83918425,
       0.36830355, 0.28380972, 0.5600429 , 0.70536968, 0.78008044,
       0.73455221, 0.55130088, 0.74358483, 0.5600429 , 0.56004

### Calculating MSE , ASE , R-squared for rbf

In [75]:
mae = mean_absolute_error(y_test, pred)
print(f"Mean Absolute Error (MAE): {mae:.2f}")

mse = mean_squared_error(y_test, pred)
print(f"Mean Squared Error (MSE): {mse:.2f}")

r2 = r2_score(y_test, pred)
print(f"R-squared (R^2): {r2:.2f}")

Mean Absolute Error (MAE): 0.39
Mean Squared Error (MSE): 0.18
R-squared (R^2): 0.21


## Linear kernel

In [78]:
reg = SVR(kernel="linear", C=100, gamma="auto")
reg.fit(x_train,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=100, gamma='auto', kernel='linear')

In [79]:
pred = reg.predict(x_test)
pred

array([ 1.18546414e+00, -1.64971162e+01, -4.05583324e+00,  2.25478413e+00,
        1.79552769e+00, -1.39698308e+01, -2.62077071e+01, -1.07458084e+01,
        1.68498581e+00, -2.21037249e+00,  5.17481867e-01, -8.92619611e+00,
       -3.74132092e+00, -5.37060445e+00, -3.58778708e-01, -1.22845102e+01,
       -4.56052148e+00,  2.59747655e+00, -7.44418684e-01, -2.01220535e+01,
       -3.92436527e-01,  1.60418199e+00, -2.25340986e+01,  2.20979317e+00,
        2.33126894e+00, -1.12626060e+00, -7.79755127e-01,  3.98048827e+00,
        7.87969463e-01, -7.83791490e+00,  2.88077616e+00, -2.50041801e-01,
        2.18234487e+00, -1.36281225e+00,  2.37193496e+00,  1.30923595e+00,
       -6.05643236e+00, -2.20766902e-01, -1.29265912e+01, -1.70967557e+00,
        2.57062361e+00, -3.02045504e+00,  2.22555231e+00,  1.45275637e+00,
        1.05611778e+00,  2.05645939e+00,  2.13278467e+00,  3.30660917e+00,
        2.23833532e+00,  1.68545067e-01, -9.79668834e+00, -1.21476203e+01,
        2.38578968e+00, -

### Calculating MSE , ASE , R-squared for linear

In [80]:
mae = mean_absolute_error(y_test, pred)
print(f"Mean Absolute Error (MAE): {mae:.2f}")

mse = mean_squared_error(y_test, pred)
print(f"Mean Squared Error (MSE): {mse:.2f}")

r2 = r2_score(y_test, pred)
print(f"R-squared (R^2): {r2:.2f}")

Mean Absolute Error (MAE): 5.23
Mean Squared Error (MSE): 96.79
R-squared (R^2): -419.26


#### Comparing the mse , ase and r-squared values for the above two kernel then we will see that  kernel 1 ie_ rbf is better than kernell 2 ie_ linear . By this we can say that rbf is better than linear. 